In [1]:
import geopandas as gpd
import fiona
import boto3
from fiona.io import ZipMemoryFile
import os
os.chdir("/Users/canyonfoot/Documents/python_proj/nwf-process-geodata")
from src.common import *

In [3]:
s3 = get_s3_client()
key = "data/raw/climate_change_indicators/flooding/fema_nfhl.zip"
BUCKET_NAME = "dataclinic-nwf"
layer_name = 'a00000015.gdbtable'


In [12]:
import boto3
import fiona
import zipfile
import io
import tempfile

def read_layer_from_zipped_gdb(s3, s3_bucket, s3_key, layer_name):

    # Download the file from S3
    response = s3.get_object(Bucket=s3_bucket, Key=s3_key)
    gdb_zip = io.BytesIO(response['Body'].read())

    # Create a temporary directory
    with tempfile.TemporaryDirectory() as tmpdirname:
        # Unzip the file into the temporary directory
        with zipfile.ZipFile(gdb_zip, 'r') as z:
            z.extractall(tmpdirname)
        
        # Find the extracted .gdb folder
        gdb_folder = [f for f in os.listdir(tmpdirname) if f.endswith('.gdb')][0]
        gdb_path = os.path.join(tmpdirname, gdb_folder) + "/" + layer_name
        print(gdb_path)
        # Read the specified layer using geopandas
        gdf = gpd.read_file(gdb_path)
    
    return gdf

read_layer_from_zipped_gdb(s3, BUCKET_NAME, key, layer_name)


/var/folders/ct/9lwc0fbn26v_v6pz1pfj1dxc0000gq/T/tmp3ehejupv/NFHL_56_20221104.gdb/a00000015.gdbtable


,DFIRM_ID,VERSION_ID,FLD_AR_ID,STUDY_TYP,FLD_ZONE,ZONE_SUBTY,SFHA_TF,STATIC_BFE,V_DATUM,DEPTH,...,AR_REVERT,AR_SUBTRV,BFE_REVERT,DEP_REVERT,DUAL_ZONE,SOURCE_CIT,GFID,SHAPE_Length,SHAPE_Area,geometry
0,56043C,1.1.1.0,56043C_1782,NP,AE,None,T,-9999.0,None,-9999.0,...,None,None,-9999.0,-9999.0,None,56043C_LOMC1,20130706BAKER2,0.000689,6.115813e-10,"MULTIPOLYGON (((-107.93081 43.99464, -107.9309..."
1,56043C,1.1.1.0,56043C_2096,NP,AE,None,T,-9999.0,None,-9999.0,...,None,None,-9999.0,-9999.0,None,56043C_LOMC1,20130706BAKER2,0.000095,1.132652e-10,"MULTIPOLYGON (((-107.94290 44.00353, -107.9429..."
2,56043C,1.1.1.0,56043C_2197,NP,AE,FLOODWAY,T,-9999.0,None,-9999.0,...,None,None,-9999.0,-9999.0,None,56043C_LOMC1,20130706BAKER2,0.024032,3.157082e-06,"MULTIPOLYGON (((-107.96563 44.01174, -107.9655..."
3,56043C,1.1.1.0,56043C_2722,NP,A,None,T,-9999.0,None,-9999.0,...,None,None,-9999.0,-9999.0,None,56043C_LOMC1,20130706BAKER2,0.016790,1.333936e-06,"MULTIPOLYGON (((-107.97745 44.00779, -107.9772..."
4,56043C,1.1.1.0,56043C_2927,NP,A,None,T,-9999.0,None,-9999.0,...,None,None,-9999.0,-9999.0,None,56043C_STUDY1,20130706BAKER2,0.330695,4.137542e-04,"MULTIPOLYGON (((-108.06275 44.03918, -108.0626..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13942,56003C,1.1.1.0,56003C_191,NP,X,AREA OF MINIMAL FLOOD HAZARD,F,-9999.0,,-9999.0,...,,,-9999.0,-9999.0,,56003C_STUDY1,58752afd-9d5f-48c8-b5c9-eed67941b88b,0.320618,2.271868e-03,"MULTIPOLYGON (((-108.51033 44.18461, -108.5103..."
13943,56003C,1.1.1.0,56003C_192,NP,X,AREA OF MINIMAL FLOOD HAZARD,F,-9999.0,,-9999.0,...,,,-9999.0,-9999.0,,56003C_STUDY1,58752afd-9d5f-48c8-b5c9-eed67941b88b,11.767967,1.765007e-01,"MULTIPOLYGON (((-108.36172 44.52880, -108.3615..."
13944,56003C,1.1.1.0,56003C_193,NP,X,AREA OF MINIMAL FLOOD HAZARD,F,-9999.0,,-9999.0,...,,,-9999.0,-9999.0,,56003C_STUDY1,58752afd-9d5f-48c8-b5c9-eed67941b88b,14.773900,1.917154e-01,"MULTIPOLYGON (((-108.36775 44.99647, -108.3674..."
13945,56003C,1.1.1.0,56003C_194,NP,X,AREA WITH REDUCED FLOOD RISK DUE TO LEVEE,F,-9999.0,,-9999.0,...,,,-9999.0,-9999.0,,56003C_LOMC1,58752afd-9d5f-48c8-b5c9-eed67941b88b,0.168710,2.862295e-04,"MULTIPOLYGON (((-108.05877 44.50486, -108.0587..."


In [ ]:
def extract_zip_file(file_to_extract, destination):
    with zipfile.ZipFile(file_to_extract, 'r') as zip_ref:
        zip_ref.extractall(destination)

In [14]:
gdf = gpd.read_file("/Users/canyonfoot/Downloads/NFHL_56_20221104 (1)/NFHL_56_20221104.gdb/a00000015.gdbtable")

In [18]:
gdf.query("FLD_ZONE.str.startswith('A')")

,DFIRM_ID,VERSION_ID,FLD_AR_ID,STUDY_TYP,FLD_ZONE,ZONE_SUBTY,SFHA_TF,STATIC_BFE,V_DATUM,DEPTH,...,AR_REVERT,AR_SUBTRV,BFE_REVERT,DEP_REVERT,DUAL_ZONE,SOURCE_CIT,GFID,SHAPE_Length,SHAPE_Area,geometry
0,56043C,1.1.1.0,56043C_1782,NP,AE,None,T,-9999.0,None,-9999.0,...,None,None,-9999.0,-9999.0,None,56043C_LOMC1,20130706BAKER2,0.000689,6.115813e-10,"MULTIPOLYGON (((-107.93081 43.99464, -107.9309..."
1,56043C,1.1.1.0,56043C_2096,NP,AE,None,T,-9999.0,None,-9999.0,...,None,None,-9999.0,-9999.0,None,56043C_LOMC1,20130706BAKER2,0.000095,1.132652e-10,"MULTIPOLYGON (((-107.94290 44.00353, -107.9429..."
2,56043C,1.1.1.0,56043C_2197,NP,AE,FLOODWAY,T,-9999.0,None,-9999.0,...,None,None,-9999.0,-9999.0,None,56043C_LOMC1,20130706BAKER2,0.024032,3.157082e-06,"MULTIPOLYGON (((-107.96563 44.01174, -107.9655..."
3,56043C,1.1.1.0,56043C_2722,NP,A,None,T,-9999.0,None,-9999.0,...,None,None,-9999.0,-9999.0,None,56043C_LOMC1,20130706BAKER2,0.016790,1.333936e-06,"MULTIPOLYGON (((-107.97745 44.00779, -107.9772..."
4,56043C,1.1.1.0,56043C_2927,NP,A,None,T,-9999.0,None,-9999.0,...,None,None,-9999.0,-9999.0,None,56043C_STUDY1,20130706BAKER2,0.330695,4.137542e-04,"MULTIPOLYGON (((-108.06275 44.03918, -108.0626..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13936,56003C,1.1.1.0,56003C_185,NP,A,None,T,-9999.0,,-9999.0,...,,,-9999.0,-9999.0,,56003C_STUDY1,58752afd-9d5f-48c8-b5c9-eed67941b88b,2.489614,1.711460e-03,"MULTIPOLYGON (((-108.37658 44.86624, -108.3766..."
13937,56003C,1.1.1.0,56003C_186,NP,A,None,T,-9999.0,,-9999.0,...,,,-9999.0,-9999.0,,56003C_STUDY1,58752afd-9d5f-48c8-b5c9-eed67941b88b,13.319293,1.522910e-02,"MULTIPOLYGON (((-108.31818 44.84530, -108.3182..."
13938,56003C,1.1.1.0,56003C_187,NP,A,None,T,-9999.0,,-9999.0,...,,,-9999.0,-9999.0,,56003C_STUDY1,58752afd-9d5f-48c8-b5c9-eed67941b88b,0.561579,1.710963e-03,"MULTIPOLYGON (((-107.92267 44.19037, -107.9225..."
13939,56003C,1.1.1.0,56003C_188,NP,A,None,T,-9999.0,,-9999.0,...,,,-9999.0,-9999.0,,56003C_STUDY1,58752afd-9d5f-48c8-b5c9-eed67941b88b,4.061318,2.718748e-03,"MULTIPOLYGON (((-107.78174 44.27912, -107.7817..."


In [19]:
gpd.read_file("https://prd-tnm.s3.amazonaws.com/StagedProducts/Hydrography/NHD/State/Shape/NHD_H_Wyoming_State_Shape.zip")

DriverError: '/vsizip/vsicurl/https://prd-tnm.s3.amazonaws.com/StagedProducts/Hydrography/NHD/State/Shape/NHD_H_Wyoming_State_Shape.zip' not recognized as a supported file format.